In [461]:
# Opracowane na podstawie https://minpy.readthedocs.io/en/latest/tutorial/rnn_mnist.html
using MLDatasets, Flux
train_data = MLDatasets.MNIST(split=:train)
test_data  = MLDatasets.MNIST(split=:test)

function loader(data; batchsize::Int=1)
    x1dim = reshape(data.features, 28 * 28, :) # reshape 28×28 pixels into a vector of pixels
    yhot  = Flux.onehotbatch(data.targets, 0:9) # make a 10×60000 OneHotMatrix
    Flux.DataLoader((x1dim, yhot); batchsize, shuffle=false)
end

rnn = RNN(196 => 10, tanh)
dense = Dense(64 => 10, identity)


Dense(64 => 10)     # 650 parameters

In [599]:
rnn_cached = deepcopy(rnn)
dense_cached = deepcopy(dense)
net = Chain(
    rnn_cached,
    # dense_cached,
)

using Statistics: mean  # standard library
function loss_and_accuracy(model, data)
    (x,y) = only(loader(data; batchsize=length(data)))
    Flux.reset!(model)
    ŷ = model(x[  1:196,:])
    ŷ = model(x[197:392,:])
    ŷ = model(x[393:588,:])
    ŷ = model(x[589:end,:])
    loss = Flux.logitcrossentropy(ŷ, y)  # did not include softmax in the model
    acc = round(100 * mean(Flux.onecold(ŷ) .== Flux.onecold(y)); digits=2)
    (; loss, acc, split=data.split)  # return a NamedTuple
end

@show loss_and_accuracy(net, test_data);  # accuracy about 10%, before training

train_log = []
settings = (;
    eta = 15e-3,
    epochs = 1,
    batchsize = 1,
)

opt_state = Flux.setup(Descent(settings.eta), net);

loss_and_accuracy(net, test_data) = (loss = 2.3676634f0, acc = 8.9, split = :test)


In [635]:
using ProgressMeter

for epoch in 1:settings.epochs
    @time for (x,y) in loader(train_data, batchsize=settings.batchsize)
        Flux.reset!(net)
        grads = Flux.gradient(model -> let
                ŷ = model(x[  1:196,:])
                # @show sum(rnn.cell.Wi)
                # @show sum(rnn.cell.Wh)
                # @show sum(rnn.cell.b)
                @show rnn_cached.state
                
                ŷ = model(x[197:392,:])
                @show rnn_cached.state
                ŷ = model(x[393:588,:])
                @show rnn_cached.state
                ŷ = model(x[589:end,:])
                @show rnn_cached.state
                final = Flux.logitcrossentropy(ŷ, y)
                @show final
                
                @show sum(rnn_cached.cell.Wi)
                @show sum(rnn_cached.cell.Wh)
                @show sum(rnn_cached.cell.b)
                # s = AccuracyModule.softmax(ŷ)
                # @show sum(x)
                # @show sum(ŷ)
                # @show sum(y)
                # @show s
                # @show y
                # @show sum(dense_cached.weight)
                # @show sum(dense_cached.bias)
                myGradient = AccuracyModule.gradient(ŷ, y)
                @show sum(myGradient)
                final
            end, net)
        # @show sum(grads[1][1][2].weight)
        # @show sum(grads[1][1][2].bias)
        # @show grads[1][1][2].bias
        @show grads[1][1][1][1].Wi
        @show grads[1][1][1][1].Wh
        @show grads[1][1][1][1].b
        Flux.update!(opt_state, net, grads[1])
    end
    
    loss, acc, _ = loss_and_accuracy(net, train_data)
    test_loss, test_acc, _ = loss_and_accuracy(net, test_data)
    @info epoch acc test_acc
    nt = (; epoch, loss, acc, test_loss, test_acc) 
    push!(train_log, nt)
end

rnn_cached.state = Float32[0.80616355; -0.99324816; 0.999962; -0.9999927; 0.99999857; -0.07662304; 1.0; -0.6067625; -0.9999995; 0.8452436;;]
rnn_cached.state = Float32[-0.97755146; -1.0; -0.9996698; 1.0; -1.0; 0.93019545; -0.994541; -0.9998532; -1.0; -1.0;;]
rnn_cached.state = Float32[-0.998499; -0.9999859; 1.0; -1.0; 1.0; -1.0; 0.9999996; -0.9999996; -0.9579484; 0.9994851;;]
rnn_cached.state = Float32[-0.98293906; -1.0; -1.0; 1.0; -1.0; -0.9765655; -0.99978566; 0.97993356; -1.0; -1.0;;]
final = 3.0977159f0
sum(rnn_cached.cell.Wi) = 10.923492f0
sum(rnn_cached.cell.Wh) = 2.1149094f0
sum(rnn_cached.cell.b) = -27.291922f0
sum(myGradient) = -3.7252903f-8
((((grads[1])[1])[1])[1]).Wi = Float32[0.0 0.0 0.0 0.0 0.0 0.0 0.0 5.23674f-5 0.0008019658 0.001841789 0.00534173 0.006294096 0.006294096 0.0058737835 0.005048726 0.0036717667 0.0008860993 -0.00030162797 -3.0334773f-5 8.763524f-5 8.763524f-5 5.984846f-5 4.1680178f-5 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.00042856426 0.0040905974 0.0

Excessive output truncated after 534874 bytes.


((((grads[1])[1])[1])[1]).Wh = Float32[-6.393657f-7 -6.387659f-7 -6.393658f-7 6.3966297f-7 -6.39629f-7 6.384468f-7 -1.79876f-7 -6.3900086f-7 6.364231f-7 6.396629f-7; -3.3821383f-5 3.3821383f-5 -3.3821383f-5 -3.3821383f-5 3.3821383f-5 2.935027f-5 3.333652f-5 -3.3821358f-5 3.3821383f-5 -3.3821383f-5; -0.0072824066 -0.0072721783 -0.007282407 0.007282393 -0.0072820066 0.007271718 -0.0020466435 -0.007278252 0.007248898 0.0072823926; -1.8356957f-8 1.8356957f-8 -1.8356957f-8 -1.8356957f-8 1.8356957f-8 1.5930208f-8 1.8093791f-8 -1.8356943f-8 1.8356957f-8 -1.8356957f-8; 2.4751162f-7 -2.4751162f-7 2.4751162f-7 2.4751162f-7 -2.4751162f-7 -2.1479113f-7 -2.4396329f-7 2.4751142f-7 -2.4751162f-7 2.4751162f-7; 0.0004843541 0.00045972486 -0.00045998604 0.00050378015 -0.0005036678 0.00044785367 -0.0005035533 -0.000368915 0.00045998592 0.00041288405; -0.0020009438 0.0020368262 -0.002037359 -0.0019996713 0.0019996758 0.0017699357 0.0019709186 -0.0020338455 0.0020373575 -0.0020031764; 1.8903174f-5 -1.8902

In [269]:
# sum(state.output) = -0.30713254f0
# sum(state.output) = 1.790454f0
# sum(state.output) = 2.16625f0
# sum(state.output) = 0.78040606f0
# loss = 2.199307f0
# sum(wr.output) = 6.487898f0
# sum(hwr.output) = 5.6139755f0
# sum(br.output) = 0.0f0
# sum(xb) = 107.94118f0
# sum(result) = 0.78040606f0
# sum(yb) = 1
# sum(gradient) = 6.7055225f-8
# sum(result - yb) = -0.21959373f0
s1 = Float32[0.16055082; 0.19607376; 0.14830309; 0.078709975; 0.081818044; 0.11087999; 0.041694667; 0.060726058; 0.057663906; 0.063579775;;]
y1 = Bool[0; 0; 0; 0; 0; 1; 0; 0; 0; 0;;]
s1 - y1
# s1 - y1 = Float32[0.16055082; 0.19607376; 0.14830309; 0.078709975; 0.081818044; -0.88912; 0.041694667; 0.060726058; 0.057663906; 0.063579775;;]

10×1 Matrix{Float32}:
  0.16055082
  0.19607376
  0.14830309
  0.078709975
  0.081818044
 -0.88912
  0.041694667
  0.060726058
  0.057663906
  0.063579775

In [270]:
# sum(rnn_cached.state) = -0.30713254f0
# sum(rnn_cached.state) = 1.790454f0
# sum(rnn_cached.state) = 2.16625f0
# sum(rnn_cached.state) = 0.78040606f0
# final = 2.199307f0
# sum(rnn_cached.cell.Wh) = 5.6139755f0
# sum(rnn_cached.cell.Wi) = 6.487898f0
# sum(rnn_cached.cell.b) = 0.0f0
# sum(x) = 107.94118f0
# sum(ŷ) = 0.78040606f0
# sum(y) = 1
s2 = Float32[0.16055082; 0.19607376; 0.14830309; 0.078709975; 0.081818044; 0.11087999; 0.041694667; 0.060726058; 0.057663906; 0.063579775;;]
y2 = Bool[0; 0; 0; 0; 0; 1; 0; 0; 0; 0;;]
s2 - y2
# sum(myGradient) = 6.7055225f-8

10×1 Matrix{Float32}:
  0.16055082
  0.19607376
  0.14830309
  0.078709975
  0.081818044
 -0.88912
  0.041694667
  0.060726058
  0.057663906
  0.063579775

In [271]:
@show s1 - s2

s1 - s2 = Float32[0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0;;]


10×1 Matrix{Float32}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [236]:
Flux.reset!(net)
x1, y1 = first(loader(train_data)); # (28×28×1×1 Array{Float32, 3}, 10×1 OneHotMatrix(::Vector{UInt32}))
y1hat = net(x1[  1:196,:])
y1hat = net(x1[197:392,:])
y1hat = net(x1[393:588,:])
y1hat = net(x1[589:end,:])
@show hcat(Flux.onecold(y1hat, 0:9), Flux.onecold(y1, 0:9))

@show loss_and_accuracy(net, train_data);

hcat(Flux.onecold(y1hat, 0:9), Flux.onecold(y1, 0:9)) = [7 5]
loss_and_accuracy(net, train_data) = (loss = 2.547189f0, acc = 8.21, split = :train)


In [681]:
include("Graph.jl")
include("DataModule.jl")
include("UtilsModule.jl")
include("AccuracyModule.jl")

Main.AccuracyModule

In [464]:
# Cache
dense_weight = deepcopy(dense.weight)
dense_bias = deepcopy(dense.bias)

rnn_weight = deepcopy(rnn.cell.Wi)
rnn_bias = deepcopy(rnn.cell.b)
rnn_h_weight = deepcopy(rnn.cell.Wh)

10×10 Matrix{Float32}:
 -0.417766   -0.15417   -0.313843  …   0.284364   -0.0572766  -0.235279
  0.144755    0.192485  -0.305884     -0.270198    0.244858    0.426091
  0.397334   -0.266666   0.27523      -0.166732    0.0807346  -0.215622
 -0.0479694  -0.529267   0.344753      0.19138    -0.386259    0.528287
  0.116263    0.292874   0.48582       0.148341   -0.116083    0.360188
  0.403403   -0.312093  -0.150956  …  -0.172287    0.0532873  -0.00578945
  0.0361245  -0.191101   0.459268      0.229905    0.139157    0.541779
 -0.0687659  -0.133807   0.397449     -0.418108    0.289734   -0.10863
  0.117244    0.410774  -0.314048     -0.355233    0.530834   -0.465246
 -0.100737    0.342372  -0.479632      0.0684645   0.240772    0.527968

In [683]:
using .DataModule, .UtilsModule, .AccuracyModule
using Random, Plots

function load_data(batch_size)
    println("Loading train data...")
    train_x, train_y = DataModule.preprocess(:train; one_hot = true)

    train_x_batched = DataModule.batch(train_x, batch_size)
    train_y_batched = DataModule.batch(train_y, batch_size)

    println("Loading test data...")
    test_x, test_y = DataModule.preprocess(:test; one_hot = true)
    return train_x, train_y, train_x_batched, train_y_batched, test_x, test_y
end

function update_weights!(graph::Vector, lr::Float64, batch_size::Int64)
    for node in graph
        if isa(node, Variable) && hasproperty(node, :gradient) && node.gradient != nothing
#             println(string("Update ", node.name, " ", sum(node.output), " ", sum(node.gradient)))
            if node.name == "state" || node.name == "x"
                node.output = node.gradient
                node.gradient = nothing
#                 println(string("UpdateSPECIAL ", node.name, " ", sum(node.output)))
            else
                # println(string("Sum1: ", node.name, " gradient: ", sum(node.gradient), " output ", sum(node.output)))
                node.gradient ./= batch_size
                node.output .-= lr * node.gradient
                # println(string("Sum2: ", node.name, " gradient: ", sum(node.gradient), " output ", sum(node.output)))
                node.gradient = nothing
            end
        end
    end
end

function reset_state!(graph::Vector)
    for node in graph
        if isa(node, Variable) && hasproperty(node, :gradient) && node.gradient != nothing
            node.gradient = nothing
        end
    end
end

function main()
    batch_size = 100
    train_x, train_y, train_x_batched, train_y_batched, test_x, test_y = load_data(batch_size)
    batches = randperm(size(train_x_batched, 1))

    epochs = 1

    x = Variable([0.], name="x")
    y = Variable([0.], name="y")

    # rnn.cell.Wi
    # rnn.cell.Wh
    # rnn.cell.b
    
    # wd = Variable(deepcopy(dense_weight), name="wd")
    # bd = Variable(deepcopy(dense_bias), name="bd")

    # wr = Variable(deepcopy(rnn_weight), name = "wr")
    # br = Variable(deepcopy(rnn_bias), name = "br")
    # hwr = Variable(deepcopy(rnn_h_weight), name = "hwr")
    # state = Variable(zeros(Float32, size(x.output)), name = "state")

    wd = Variable(UtilsModule.glorot_uniform(10, 64), name="wd")
    bd = Variable(UtilsModule.glorot_uniform(10, ), name="bd")

    wr = Variable(UtilsModule.glorot_uniform(64, 196), name = "wr")
    br = Variable(UtilsModule.glorot_uniform(64, ), name = "br")
    hwr = Variable(UtilsModule.glorot_uniform(64, 64), name = "hwr")
    state = Variable(zeros(Float32, size(x.output)), name = "state")


    r = rnn_layer(x, wr, br, hwr, state)
    d = dense_layer(r, wd, bd)
    graph = topological_sort(d)

    batch_loss = Float64[]
    println("Training")
    for epoch in 1:epochs
        @time for (xb,yb) in loader(train_data, batchsize=100)
            reset_state!(graph)
            y.output = yb
            x.output = xb[  1:196,:]
            state.output = zeros(Float32, 64, size(x.output, 2))

            forward!(graph)
            # @show sum(wr.output)
            # @show sum(hwr.output)
            # @show sum(br.output)
            # @show sum(state.output)

            x.output = xb[197:392,:]
            forward!(graph)
            # @show sum(state.output)

            x.output = xb[393:588,:]
            forward!(graph)
            # @show sum(state.output)

            x.output = xb[589:end,:]
            result = forward!(graph)
            # @show sum(state.output)

            loss, acc, _ = AccuracyModule.loss_and_accuracy(result, yb)
            gradient = AccuracyModule.get_gradient(result, yb)

            push!(batch_loss, loss)
            backward!(graph, seed=gradient)

            # @show loss

            # @show sum(wr.output)
            # @show sum(hwr.output)
            # @show sum(br.output)
            
            # @show sum(xb)
            # @show sum(result)
            # @show sum(yb)
            
            # @show sum(gradient)
            # @show sum(result - yb)
            # s = AccuracyModule.softmax(result)
            # @show s
            # @show yb
            # @show s - yb

            # @show sum(wd.output)
            # @show sum(bd.output)

            # @show sum(wd.gradient)
            # @show sum(bd.gradient)
            # @show bd.gradient
            # @show sum(gradient)
            # @show sum(wr.gradient)
            # @show sum(hwr.gradient)
            # @show sum(br.gradient)
            # Update gradientu raczej na samym końcu jak w Fluxie
            update_weights!(graph, 15e-3, batch_size)
            # reset_state!(graph_optimizer)
        end
        reset_state!(graph)

        test_graph = topological_sort(d)

        y.output = test_y
        x.output = test_x[  1:196,:]
        state.output = zeros(Float32, 64, size(x.output, 2))
        reset_state!(test_graph)
        forward!(test_graph)

        x.output = test_x[197:392,:]
        forward!(test_graph)

        x.output = test_x[393:588,:]
        forward!(test_graph)

        x.output = test_x[589:end,:]
        result = forward!(test_graph)

        loss, acc, _ = AccuracyModule.loss_and_accuracy(result, test_y)

        state.output = zeros(Float32, size(x.output))
        @show epoch loss acc
    end
    plot(batch_loss, xlabel="Batch num", ylabel="loss", title="Loss over batches")
end


main (generic function with 1 method)

In [684]:
main()

Loading train data...
Loading test data...
Training


LoadError: DimensionMismatch: cannot broadcast array to have fewer non-singleton dimensions

In [44]:
sum(rnn_cached.state) = -1.2453158f0
sum(rnn_cached.state) = -1.9936836f0
sum(rnn_cached.state) = -6.1363106f0
sum(rnn_cached.state) = -2.287208f0
final = 2.0137506f0
sum(rnn_cached.cell.Wi) = -9.323987f0
sum(rnn_cached.cell.Wh) = 6.531007f0
sum(rnn_cached.cell.b) = 0.0f0
sum(myGradient) = -1.15484f-7
sum(((((grads[1])[1])[1])[1]).Wi) = -7.3526535f0
sum(((((grads[1])[1])[1])[1]).Wh) = 6.627337f0
sum(((((grads[1])[1])[1])[1]).b) = -0.7453849f0

10×64 Matrix{Float32}:
  0.0665468  -0.143778   -0.129514   …  -0.273826   -0.078714    -0.19697
 -0.0800066  -0.0729043   0.197498      -0.152116   -0.0580736   -0.18379
  0.0983914   0.214602    0.19418       -0.271145    0.0232201   -0.0162141
 -0.103123    0.231914   -0.139903      -0.0811338  -0.018343    -0.0645869
  0.237592   -0.248433   -0.252619       0.0389151   0.021613     0.17853
 -0.272307   -0.277688   -0.107417   …  -0.192654    0.106212    -0.205456
 -0.18117     0.0724456  -0.0443333      0.264246   -0.164543     0.171745
  0.0332814   0.27214    -0.119537      -0.168845   -0.183243    -0.189063
 -0.272497    0.0588258  -0.258422      -0.0777486   0.250726     0.188828
 -0.0543392  -0.0136128   0.125465      -0.234033   -0.00481401   0.146143

In [602]:
# TARGETS
# sum(((((grads[1])[1])[1])[1]).Wi) = -7.3526535f0
# sum(((((grads[1])[1])[1])[1]).Wh) = 6.627337f0
# sum(((((grads[1])[1])[1])[1]).b) = -0.7453849f0

grad_wi = Float32[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0730086 0.37160116 0.42571086 0.5342862 0.5638305 0.5638305 0.2745692 -0.29323995 -0.380288 -0.4667052 -0.049504116 0.049289152 0.12217765 0.12217765 0.0834384 0.058108885 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.013294942 0.15312143 0.4880588 0.5638305 0.5638305 0.5638305 0.5638305 0.52099127 0.19340223 -0.3864074 -0.5354722 -0.5444121 -0.5356691 -0.09642043 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.04062343 0.12704055 0.16692537 0.18686779 0.3060655 0.41930333 0.3462947 0.557183 0.47519758 0.31356883 0.016389685 -0.057138037 -0.26804608 -0.67046845 -0.90349513 -0.6678008 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.10045067 0.18686779 0.18686779 0.18686779 0.15658486 0.12057167 0.09898621 0.24127331 0.37696275 0.13409743 0.0 0.0 0.0 -0.8892107 -0.90349513 -0.8892107 -0.22855215 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.20710602 0.37696275 0.28309456 -0.16129191 -0.4642465 -0.6535163 -0.90349513 -0.90349513 -0.73922336 -0.0071422546 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.016389685 0.1436485 -0.15259658 -0.7145226 -0.90452766 -0.9107227 -0.91129625 -0.90282 -0.65143657 -0.009521991 -0.014627154 -0.0141682625 -0.0072848964 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0017208416 -0.0020650101 -0.091099024 -0.41594216 -0.7468215 -0.5589245 -0.582764 -0.67961216 -0.7570906 -0.730819 -0.29145423 -0.009866159 -0.014512431 -0.013881456 -0.011185471 -0.003671129 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.029412793 0.17448643 0.24261397 0.44473436 0.4997331 0.4997331 0.76572764 1.2878652 1.2549324 1.0940611 0.5441977 0.13792138 0.04922141 0.04922141 0.03361462 0.023410182 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.024749435 0.24592432 0.43257535 0.4997331 0.4997331 0.4997331 0.4997331 0.41998494 0.376624 0.7324246 0.9790871 0.97548556 0.4925826 0.08866487 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.075623274 0.2364946 0.3107429 0.34786704 0.39588794 0.4415078 0.41209498 0.48735836 0.33473688 0.13817817 0.006602872 0.052542143 0.33121243 0.91997886 0.83082265 0.6140863 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.18699574 0.34786704 0.34786704 0.34786704 0.29149336 0.19402441 0.18209612 0.1144397 0.15186605 0.054023497 0.0 0.0 0.0 0.81768703 0.83082265 0.81768703 0.21016857 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.083436295 0.15186605 0.114049606 0.15225919 0.42690495 0.6009508 0.83082265 0.83082265 0.679764 0.006567768 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.006602872 0.24240732 0.6395984 0.79574525 0.83254015 0.84284526 0.8437995 0.8376691 0.6001477 0.015839336 0.02433151 0.023568168 0.012118046 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0028625305 0.0034350369 0.08778247 0.3890571 0.76296854 0.9996262 0.99002206 0.9510051 0.9197915 0.6848016 0.27761194 0.016411843 0.024140675 0.02309108 0.018606449 0.0061067324 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0152872605 -0.05987939 -0.037687846 0.054174528 0.07917109 0.07917109 0.109106004 0.1678671 0.1383012 0.06518623 -0.03307995 -0.019775428 -0.025582764 -0.025582764 -0.017471155 -0.012167412 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.011248455 0.10124459 0.06853151 0.07917109 0.07917109 0.07917109 0.07917109 0.04292608 0.0048506223 0.017582374 0.016440095 0.018312009 0.055435 0.0099783 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.034370277 0.107485235 0.1412306 0.15810327 0.13314448 0.10943363 0.12472089 0.07354687 0.0041814074 -0.057082854 -0.003431834 0.0059130676 0.020954348 0.04508512 0.09350037 0.06910897 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.084988326 0.15810327 0.15810327 0.15810327 0.1324818 0.079995625 0.082176685 -0.038047053 -0.07893218 -0.028078642 0.0 0.0 0.0 0.092022106 0.09350037 0.092022106 0.02365227 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.043365903 -0.07893218 -0.059277136 0.016376097 0.048043672 0.06763071 0.09350037 0.09350037 0.076500304 0.00073913345 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.003431834 -0.048397858 -0.045439247 0.041588888 0.07229743 -0.054920185 -0.066699594 -0.113747984 0.036634672 -0.1955382 -0.30037493 -0.2909514 -0.1495985 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.035338227 -0.042405874 -0.10185685 -0.1392723 -0.12949519 -0.279707 -0.2747153 -0.25443625 -0.23821306 -0.22404814 -0.2362105 -0.20260584 -0.29801905 -0.2850617 -0.22969848 -0.07538822 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.041476052 0.2093839 0.23690405 0.28757942 0.3013686 0.3013686 0.22104752 0.06338021 0.03152901 -0.0088044405 0.095145844 0.053929396 0.06940891 0.06940891 0.047401205 0.033011552 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0062051443 0.07418497 0.26086846 0.3013686 0.3013686 0.3013686 0.3013686 0.28137425 0.15055272 -0.027284525 -0.041806266 -0.04688497 -0.14874275 -0.026773693 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.018960161 0.059293598 0.07790903 0.08721674 0.15493277 0.21926296 0.17778692 0.29826605 0.26000097 0.17731428 0.0093109505 -0.015865894 -0.05582315 -0.119534485 -0.25087944 -0.18543261 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.04688331 0.08721674 0.08721674 0.08721674 0.073082805 0.05838888 0.046350837 0.135869 0.21415186 0.07618051 0.0 0.0 0.0 -0.24691296 -0.25087944 -0.24691296 -0.063463576 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.117656566 0.21415186 0.16082552 -0.04392154 -0.12891039 -0.18146615 -0.25087944 -0.25087944 -0.20526499 -0.0019832368 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0093109505 0.12335425 0.07460342 -0.160618 -0.24366835 -0.20040187 -0.19639571 -0.17779684 -0.17005853 0.06650219 0.10215698 0.098952055 0.050878182 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.012018468 0.014422162 0.01385903 -0.051349673 -0.121417254 0.054470785 0.04092758 -0.014091663 -0.058107078 -0.09711309 0.0127922855 0.06890589 0.101355754 0.09694898 0.078120045 0.025639402 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.004751197 -0.019664323 -0.01473809 0.007074926 0.013010442 0.013010442 -0.08211218 -0.26883435 -0.29226226 -0.30962366 -0.16378903 -0.038376406 -0.007950983 -0.007950983 -0.00542994 -0.0037815652 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.002670982 0.023628991 0.011262005 0.013010442 0.013010442 0.013010442 0.013010442 0.004403945 -0.060173586 -0.23474151 -0.32106128 -0.3204795 -0.17615299 -0.03170754 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.008161333 0.025522716 0.033535663 0.037542135 0.029785078 0.022415873 0.02716707 0.011674952 -0.0047961026 -0.01824519 -0.0010665953 -0.018789653 -0.11338428 -0.31086934 -0.2971114 -0.21960405 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.020180753 0.037542135 0.037542135 0.037542135 0.031458233 0.01867488 0.019490238 -0.012558129 -0.024531692 -0.008726689 0.0 0.0 0.0 -0.29241395 -0.2971114 -0.29241395 -0.07515861 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.013477886 -0.024531692 -0.01842301 -0.054214172 -0.15266593 -0.21490666 -0.2971114 -0.2971114 -0.24309114 -0.0023487066 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0010665953 -0.06475679 -0.20154 -0.27891833 -0.30031538 -0.3195395 -0.3213195 -0.32473844 -0.21836032 -0.029548137 -0.045390207 -0.0439662 -0.022606103 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0053400244 -0.0064080297 -0.044916555 -0.1612884 -0.29318592 -0.3655137 -0.3639623 -0.3576597 -0.35261762 -0.28117618 -0.13164973 -0.03061614 -0.045034207 -0.0430762 -0.034710158 -0.011392052 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.061282184 0.0023447275 -0.5309998 -2.418426 -2.9320111 -2.9320111 -2.861404 -2.7228045 -2.0052924 -0.5030554 0.39196733 0.13810347 0.10255386 0.10255386 0.070036784 0.048775617 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.23111343 -2.1730657 -2.537986 -2.9320111 -2.9320111 -2.9320111 -2.9320111 -2.1873126 -0.7403145 0.2741981 0.5294511 0.52194715 0.13075428 0.02353577 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.7061799 -2.2084172 -2.9017575 -3.2484274 -3.148375 -3.0533252 -3.1146073 -2.8164544 -1.3912553 0.11514866 0.013757225 0.013947124 0.13484588 0.41226834 0.22053887 0.16300699 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.7461903 -3.2484274 -3.2484274 -3.2484274 -2.7220025 -1.7158415 -1.6935811 -0.012832999 0.31641617 0.11255912 0.0 0.0 0.0 0.21705209 0.22053887 0.21705209 0.055788495 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.1738413 0.31641617 0.2376248 0.04259929 0.113320366 0.15952021 0.22053887 0.22053887 0.1804409 0.0017433905 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.013757225 0.27144653 0.4443809 0.286118 0.2194927 0.21321577 0.21263456 0.2077528 0.1571374 -0.009647907 -0.014820579 -0.01435562 -0.0073812297 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0017435976 -0.0020923172 0.015457412 0.09042278 0.22653724 0.5072428 0.4872323 0.40593964 0.3409055 0.16175704 0.054915242 -0.0099966265 -0.014704339 -0.014065021 -0.011333385 -0.0037196751 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.06285804 -0.30181965 -0.31453562 -0.2922247 -0.28615373 -0.28615373 -0.22806066 -0.1140261 -0.11305014 -0.13080777 -0.2363369 -0.10137202 -0.105191 -0.105191 -0.07183775 -0.050029866 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0027319426 0.002862746 -0.24769828 -0.28615373 -0.28615373 -0.28615373 -0.28615373 -0.29495668 -0.20958188 -0.09870781 -0.13540457 -0.12770768 0.10757977 0.019364359 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.008347602 0.026105229 0.034301054 0.03839897 -0.0642264 -0.16172048 -0.098862454 -0.2875197 -0.3043667 -0.26130795 -0.014110987 0.011475176 0.011538325 -0.016819812 0.1814512 0.13411611 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.020641344 0.03839897 0.03839897 0.03839897 0.03217621 0.0025301315 0.01875143 -0.19512543 -0.3245527 -0.115453534 0.0 0.0 0.0 0.17858241 0.1814512 0.17858241 0.045900702 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.17831157 -0.3245527 -0.24373524 0.030425496 0.093235806 0.13124731 0.1814512 0.1814512 0.14846008 0.001434397 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.014110987 -0.21709651 -0.22639939 0.06644919 0.17345914 0.12550674 0.121066704 0.10159897 0.11898603 -0.073704615 -0.11322094 -0.10966891 -0.056388468 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.013320111 -0.015984133 -0.024523582 0.013384055 0.03812164 -0.24004062 -0.21951553 -0.13613245 -0.06942596 0.030541137 -0.043959342 -0.07636864 -0.11233293 -0.10744889 -0.086580716 -0.028416237 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.054462284 -0.22076777 -0.15562156 0.12408302 0.20019308 0.20019308 0.21904147 0.2560401 0.15441383 -0.06820815 -0.24748392 -0.097549796 -0.09114096 -0.09114096 -0.06224261 -0.043347534 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.03424954 0.30536407 0.17328966 0.20019308 0.20019308 0.20019308 0.20019308 0.089833476 -0.0573806 -0.14188224 -0.21566226 -0.20899339 0.03490442 0.006282795 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.10465137 0.3272734 0.430022 0.4813963 0.3924783 0.30800614 0.36246845 0.18306834 -0.028137147 -0.20692211 -0.012226227 0.003723138 -0.026152695 -0.112527765 0.05887212 0.043514177 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.2587743 0.4813963 0.4813963 0.4813963 0.40338346 0.24131091 0.25005183 -0.14072314 -0.2812032 -0.10003277 0.0 0.0 0.0 0.057941332 0.05887212 0.057941332 0.014892552 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.15449506 -0.2812032 -0.21118028 0.00848107 0.030250497 0.04258339 0.05887212 0.05887212 0.0481681 0.00046539225 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.012226227 -0.20200844 -0.24618399 -0.023935657 0.05945231 0.06293345 0.06325579 0.063814774 0.043188747 0.0053506494 0.008219371 0.007961508 0.004093569 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0009669848 0.0011603818 0.008614592 0.031491213 0.018003792 -0.20083848 -0.1830549 -0.110809006 -0.053012297 0.05381535 0.025402684 0.0055440464 0.008154905 0.007800344 0.006285401 0.002062901 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.05591262 0.31761143 0.42079455 0.7150236 0.79508597 0.79508597 0.84182775 0.9335804 0.8309965 0.5968143 0.35908288 0.120546386 0.093568064 0.093568064 0.063900135 0.044501882 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.036028046 0.36280578 0.6882364 0.79508597 0.79508597 0.79508597 0.79508597 0.67899555 0.41202503 0.33302277 0.4278416 0.42099518 0.0865591 0.015580636 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.110085696 0.344268 0.45235214 0.5063942 0.5976801 0.68440175 0.62848914 0.77707183 0.554899 0.2559373 0.012551813 0.00923297 0.102732815 0.32114467 0.14599633 0.10791034 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.2722119 0.5063942 0.5063942 0.5063942 0.42433032 0.28618538 0.26534677 0.20775032 0.2886917 0.10269665 0.0 0.0 0.0 0.1436881 0.14599633 0.1436881 0.03693188 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.15860929 0.2886917 0.21680404 0.028826937 0.075017884 0.1056021 0.14599633 0.14599633 0.11945155 0.0011541212 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.012551813 0.23941678 0.3747409 0.2126663 0.14664057 0.15050593 0.15086384 0.15052849 0.105955586 0.00594122 0.009126573 0.008840249 0.004545391 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0010737145 0.0012884574 0.01721376 0.07129665 0.17355238 0.43005013 0.41179293 0.33762318 0.2782873 0.12618525 0.05306359 0.006155963 0.009054992 0.008661296 0.006979144 0.002290591 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.04015877 0.20812508 0.24485043 0.32837322 0.3511005 0.3511005 0.3429367 0.3269114 0.29435113 0.2278738 0.19485384 0.07370001 0.067204475 0.067204475 0.045895737 0.031963103 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.010227282 0.11191137 0.30391702 0.3511005 0.3511005 0.3511005 0.3511005 0.31814593 0.20319334 0.13552822 0.17693362 0.17201622 -0.015118195 -0.002721275 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.031250026 0.09772735 0.12840919 0.14375012 0.20931545 0.27160254 0.23144375 0.34598684 0.28291863 0.17426118 0.009015234 -0.0016126074 0.025868088 0.100814715 -0.025499353 -0.01884735 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.077272795 0.14375012 0.14375012 0.14375012 0.12045464 0.088178545 0.075819634 0.13530421 0.20735039 0.07376101 0.0 0.0 0.0 -0.025096204 -0.025499353 -0.025096204 -0.0064504296 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.11391978 0.20735039 0.15571769 -0.0029971125 -0.013102436 -0.018444195 -0.025499353 -0.025499353 -0.020863108 -0.00020157592 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.009015234 0.15134624 0.1897895 0.031644963 -0.028143626 -0.044009257 -0.0454783 -0.05090519 -0.022162914 -0.024386065 -0.03746052 -0.03628529 -0.018656809 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.00440712 -0.0052885446 -0.016227888 -0.034113046 -0.018562984 0.1348495 0.12173644 0.06846461 0.025847139 -0.05660553 -0.040914863 -0.02526749 -0.037166715 -0.03555077 -0.028646281 -0.009401857 0.0 0.0 0.0 0.0]
grad_wh = Float32[-0.429739 -0.18078123 0.09092824 0.030082181 0.6690924 0.4975047 0.11652219 -0.74249345 0.08096274 0.054552093; 0.13584612 0.29206663 -0.62276804 -0.8211795 -0.8747562 -0.858977 -0.6115321 0.2365894 -0.63250196 -0.2543896; 0.018191548 -0.038268957 -0.14922012 -0.1343296 -0.2052588 -0.2056132 -0.11917053 -0.041084554 -0.103297874 -0.08859931; -0.15935442 -0.0056934394 -0.027327958 -0.09584829 0.18521199 0.11698623 -0.02534958 -0.24888393 -0.04985006 0.006022961; -0.092184186 -0.12495613 0.0811598 0.12988326 0.20362934 0.17188603 0.10042147 -0.20307493 0.109432265 0.019431645; 0.67175746 0.92410916 2.5671384 2.5686533 2.2213168 2.6352336 2.0016277 2.3608732 1.8355632 1.4643499; 0.14373164 -0.09830366 -0.029255642 0.09191013 -0.24488229 -0.18467054 0.0025669523 0.13661674 0.04858509 -0.061039537; -0.0019615144 -0.21595576 -0.36414564 -0.27014643 -0.4547996 -0.47295234 -0.26042932 -0.25578818 -0.20305209 -0.24346328; -0.14939183 0.08060404 -0.5365794 -0.71438587 -0.41329235 -0.50832117 -0.48845047 -0.31365782 -0.51413965 -0.23006772; -0.100155264 0.058808006 -0.15455194 -0.25306088 -0.03583848 -0.08775349 -0.15089643 -0.15030453 -0.17547807 -0.048855774]
grad_b = Float32[-0.3569769, 1.3654056, -0.12633847, 0.1530453, -0.33173698, -2.7477274, -0.21875116, 0.2693325, 0.95764816, 0.29071456]

10-element Vector{Float32}:
 -0.3569769
  1.3654056
 -0.12633847
  0.1530453
 -0.33173698
 -2.7477274
 -0.21875116
  0.2693325
  0.95764816
  0.29071456

In [520]:
x = Float32[0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.09019608; 0.25882354; 0.8352941; 0.99215686; 0.99215686; 0.99215686; 0.99215686; 0.7764706; 0.31764707; 0.007843138; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.07058824; 0.67058825; 0.85882354; 0.99215686; 0.99215686; 0.99215686; 0.99215686; 0.7647059; 0.3137255; 0.03529412; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.21568628; 0.6745098; 0.8862745; 0.99215686; 0.99215686; 0.99215686; 0.99215686; 0.95686275; 0.52156866; 0.043137256; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.53333336; 0.99215686; 0.99215686; 0.99215686; 0.83137256; 0.5294118; 0.5176471; 0.0627451; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0;;]
w = Float32[0.14515355 0.0883916 -0.069068804 0.11364511 -0.1477951 -0.13848603 0.0107065 0.13735983 -0.0138689885 0.040974326 0.15517676 0.11890221 0.15801781 0.16872783 0.096594036 0.06477464 0.04840683 -0.07921544 0.16034082 0.06375075 0.0064423652 -0.06811549 0.12940466 -0.07720306 -0.020166926 -0.014807552 0.09211925 0.027524723 0.10923556 0.16954945 -0.16470589 -0.109897174 0.1002592 0.1492653 0.018350324 -0.04477104 -0.046654556 -0.098970175 0.11200798 0.11612313 0.0350407 0.02072187 0.01823204 -0.1264039 -0.15625167 -0.071893506 -0.1151248 0.14835614 -0.14428222 0.020799525 -0.040571988 0.026434468 -0.13700062 -0.16205618 -0.061191604 -0.0028139825 -0.08226554 -0.10010535 0.085893005 -0.04252958 0.046221092 0.039494857 -0.019497931 -0.09243614 -4.66098f-5 0.14182259 -0.013799796 -0.019901346 -0.035147633 0.09113491 -0.093467616 0.012272293 -0.08343032 -0.012842393 0.13132901 -0.17018951 -0.015389147 0.02928796 0.1488639 -0.08274696 -0.13000001 0.01282923 -0.117652774 0.14574529 0.14401282 -0.015623295 -0.05916079 -0.13152735 -0.097412564 -0.16842209 0.09972403 0.028290011 0.16501682 0.089883424 -0.1536078 -0.02820497 0.026966605 -0.030883905 0.15585408 -0.0661463 -0.16660485 0.10780622 -0.019635053 -0.07863252 0.10377162 0.12245881 -0.16673674 0.1618623 -0.1024334 0.061749782 0.13228464 0.13327992 -0.05376659 -0.048846845 0.0670517 0.011470751 0.14460522 0.0948319 0.064786196 0.12581797 0.13774286 0.078071535 0.14973845 -0.11796317 -0.003359893 -0.04178136 -0.16256152 0.017833019 -0.15122998 -0.051316146 -0.07743012 -0.07603836 -0.036485687 -0.1333052 0.11256506 -0.052133963 0.11777674 0.0035680605 -0.15412748 -0.1566233 -0.03168323 0.09204135 0.04677457 -0.10939415 -0.018997287 0.040165886 0.021788992 0.08177806 -0.12083176 0.14378667 -0.11232609 -0.028796533 -0.12201411 -0.13513835 0.12617105 0.022562174 -0.056498867 0.052835084 0.14733967 -0.033921372 -0.15011644 0.10009453 -0.06519746 0.106504835 0.118786685 0.08907163 -0.11490741 0.044999897 -0.100386724 0.098406956 -0.1450022 0.072316535 0.14505821 -0.11367845 -0.0019934182 -0.04975418 -0.101717085 -0.012966272 0.014754879 0.030544044 0.03691496 -0.13543426 0.10435372 -0.13692182 -0.10623736 -0.14648212 -0.09333149 0.08867893 -0.020256769 0.00797172 0.07010671 -0.11330598 0.14484994 0.09152095 0.0642108 -0.15452212; 0.026706254 -0.061977074 -0.10692119 -0.06020348 -0.07918008 -0.13095573 0.017901622 -0.11434425 -0.07749293 0.048965026 0.11609735 0.05467124 -0.008545523 0.0491641 -0.07292527 -0.067951426 -0.04828474 -0.020789068 0.034178145 0.030312257 -0.010569011 -0.11928627 0.09356684 -0.027121125 -0.062410884 -0.12940606 0.16040765 0.15131326 0.064241976 0.16047129 0.021885222 0.0729648 -0.009502641 0.08113482 -0.106974326 -0.085928835 0.018721778 0.016687691 0.053809352 -0.1423391 -0.025257608 0.14922361 0.08983767 -0.11691521 0.059147507 -0.024771022 -0.15545891 -0.1431832 0.10373192 -0.046938486 0.018028714 -0.09211715 0.035951048 -0.030900301 -0.10365081 0.1406685 -0.025628818 0.13129425 0.0007412402 0.15342438 -0.013605137 0.07417521 0.10626145 0.07581888 0.017853059 0.15743083 -0.046055302 -0.060904864 0.08220495 0.13720301 -0.16299616 -0.025488643 -0.0035022248 0.16090396 -0.009998503 -0.036303297 -0.066016175 0.014049283 -0.10676433 -0.047191903 -0.03145797 -0.09313187 0.100051396 0.036896568 -0.14366968 -0.16698524 0.009829398 -0.10839211 0.16338645 -0.054734185 0.05397158 0.026084255 -0.14436074 -0.096308455 -0.115336485 0.15422717 0.019852275 0.038914237 -0.13581045 -0.060237214 0.019338366 0.156245 -0.011474982 -0.075189315 0.043415003 -0.018819353 -0.017667819 0.035132576 -0.076529786 -0.050324827 0.012900294 0.018879348 -0.008530265 -0.14655922 0.05199082 -0.16261937 0.043283697 -0.011480638 0.11482603 -0.035138275 0.14390297 -0.15176812 0.065360956 0.026861077 -0.013031253 0.06360885 -0.043483544 0.080980465 0.043016143 0.003758487 0.01920462 -0.14717637 -0.07346339 -0.038669206 -0.03495452 0.03625339 -0.05388886 0.113089465 -0.045593925 0.13029219 0.06771209 -0.06037159 -0.0055474816 0.06040705 0.08472947 -0.059720356 -0.08859575 0.05877937 -0.03821627 -0.15246885 0.11014495 0.034218997 -0.118187755 -0.056952696 0.15267144 -0.10262623 0.11392291 0.16693127 -0.018367657 -0.04617316 0.04332219 0.09521857 0.03443215 -0.07824654 0.117577866 0.09442459 0.14589314 0.09976995 -0.101436995 -0.14092769 0.055411257 0.15530732 0.019213593 0.04263645 -0.0057049296 0.10147132 -0.07550246 0.014079576 -0.16695863 0.14634074 0.009430844 0.11602849 -0.08851451 0.10298751 -0.0830328 0.058196146 -0.12909935 0.11906687 0.04050129 0.15693612 -0.008385023 -0.046274982 0.13097353 -0.14279303 0.04375779 -0.09647862; 0.046298094 0.15842845 -0.08979134 0.04461221 -0.01129603 0.13405298 -0.033026814 0.1653445 -0.1497269 -0.10395934 0.02900759 -0.080372974 0.10273131 -0.12195072 0.1343421 -0.14767332 -0.10570618 0.026279422 -0.0754056 -0.15876146 0.010867753 -0.0942961 0.09861923 0.045416273 -0.11833442 0.08810116 -0.0295688 0.11638153 -0.09799534 -0.12766615 -0.1436719 0.054764986 -0.088910416 -0.07700838 -0.055235397 0.00088222924 -0.11656146 0.045006957 -0.03263164 0.15387306 0.14864317 -0.1419399 0.13160807 -0.07781009 -0.15899806 -0.04932039 -0.015854493 -0.11569294 -0.08547787 0.018090643 -0.066729344 0.11663486 0.1501577 -0.093976155 -0.11387832 -0.12729608 0.13646738 0.08145171 -0.032003984 -0.11563756 0.1074369 -0.15177569 0.088293485 -0.038509212 -0.16455398 -0.039317288 -0.049170326 0.10192387 0.03630362 0.0053038937 -0.08118963 -0.15663864 -0.14137204 -0.1287423 -0.09258307 0.08039022 -0.12993367 0.14006343 0.016865423 -0.021585198 -0.11220862 0.0006529237 -0.017403277 0.14622697 0.14899665 -0.01140876 -0.08618512 -0.15510115 0.15315706 -0.010390628 0.03855503 0.010762855 -0.005797559 -0.086223915 0.16821985 0.032042295 0.08254182 0.028397512 0.13160755 -0.0054113953 0.04053494 0.088605635 -0.06078593 -0.08583832 -0.018692624 0.15428634 0.07039121 0.019270943 -0.08299644 -0.14160043 -0.15979728 -0.065107964 0.13108061 -0.113663636 0.14249958 0.0033143207 0.16294935 -0.034585793 -0.10894638 -0.07296228 0.06938494 -0.105161875 -0.03252898 -0.015698712 -0.019430386 0.11775002 0.0109272 -0.12747116 0.16461024 0.15013558 0.10985636 -0.0944144 0.08556196 -0.07814489 -0.08686379 -0.15596925 0.108961314 0.035933692 -0.13295332 -0.011863668 0.10778348 -0.16554816 -0.049470168 -0.019373786 -0.14083453 0.06428946 0.019815044 -0.15001398 -0.14039601 -0.12781128 -0.13708869 -0.118361644 0.1552226 0.13154498 0.04949686 0.14706172 0.12647897 0.12096046 -0.035566878 -0.14729159 0.049686756 0.01996533 -0.018391645 0.16467626 0.0026932976 0.13834934 -0.015413765 -0.043877337 -0.09968216 0.089800745 -0.1694888 0.054227315 -0.13770184 -0.09313541 -0.03626444 -0.0030222721 0.017111309 -0.02044447 0.12279267 -0.03151561 -0.08754748 -0.15791652 0.0752263 0.018074572 -0.16027248 -0.10272002 0.13679168 -0.033776887 0.053787034 -0.15634876 -0.16590872 0.16243039 -0.006601949 -0.029453607 -0.04722541 0.041759938; 0.07458855 0.16677816 -0.16523412 0.059519876 0.018525839 0.10687967 -0.12276427 0.048713565 0.103348166 -0.051628843 -0.077405386 -0.08998793 0.13279794 0.0599875 -0.0120576965 -0.056870665 -0.090221 0.095682345 0.01416415 0.038127914 0.15808828 -0.12482655 0.13209796 0.12066471 0.14292413 -0.17037863 0.06387068 0.0014693377 0.16751997 -0.019060172 -0.027274096 -0.02415891 -0.068302944 0.11212908 -0.018402101 -0.06989923 0.08893937 0.08735486 -0.077283725 0.09938289 -0.114332244 -0.14831533 -0.09153279 0.14466806 -0.07983681 -0.11015633 0.14052889 0.049071003 0.05377131 0.095083356 -0.10471244 -0.082497895 -0.03521068 0.13131365 0.016068295 0.09976228 0.0051782653 0.14799298 -0.14903446 -0.15401977 0.06902083 -0.11303716 -0.1675524 -0.16716006 -0.050982837 -0.014277633 0.1557577 -0.016378207 0.01905002 0.10881396 -0.16152409 0.17065135 0.16636702 -0.1706548 0.0021460648 0.059761573 0.10848128 0.051567912 0.11503428 0.0862738 -0.09650065 0.062521644 -0.019480657 -0.11408028 0.12634496 -0.10862508 0.14811806 -0.10258078 0.1198168 -0.009645075 -0.07093108 -0.045232397 0.077396676 -0.036853313 -0.11017795 -0.09458424 -0.0009866995 -0.123581655 0.03302779 0.04911969 0.07737619 -0.053416822 -0.11942716 -0.020823166 -0.10882452 0.07854567 0.1542211 -0.11219963 -0.10255787 -0.025375383 -0.11919525 0.16758743 -0.023582524 0.1578895 -0.15791872 0.16150519 0.09083538 0.14107923 0.06754691 -0.14133008 0.11212348 -0.09857899 -0.13459665 -0.05841174 0.1351738 0.13520621 -0.07731347 0.026168788 0.07129419 0.14359555 -0.14025602 -0.08518277 0.100719884 0.02826051 -0.073924705 -0.07894243 0.055792496 -0.07892231 -0.14657868 -0.13227616 -0.078177206 0.1553905 -0.11097506 -0.13453573 -0.00038532936 -0.079235256 -0.061773587 -0.15249814 -0.036643296 -0.098612495 -0.11008188 -0.011656661 -0.13083053 -0.127947 -0.15337144 -0.114430636 0.09235824 -0.03207869 -0.083756626 0.0015597905 -0.043419316 -0.13560967 -0.16463317 -0.09886713 -0.055480022 0.068189114 -0.018610574 -0.13963348 -0.09284434 -0.06464972 -0.12617595 -0.03684652 -0.021956734 0.03502229 0.08534677 0.0068874876 -0.0684738 -0.12122295 0.16068181 -0.0037755156 -0.12945852 -0.034260176 -0.06898834 -0.1189495 -0.09289241 0.13361959 -0.014026518 -0.09750234 -0.019638736 -0.023081636 0.034774102 -0.13504317 0.02178712 0.1039109 0.122334406 -0.094809964; -0.13596906 -0.044036616 0.110493846 -0.11624972 0.059807185 -0.11722254 -0.0012537446 0.11287225 0.08585795 0.014132737 0.118253104 -0.16801287 -0.07260834 -0.021870961 -0.07596341 -0.032011714 0.008046161 0.1208374 -0.043423466 -0.07559558 0.16204913 0.09532155 0.14420031 0.03829551 0.09940645 -0.16089733 0.06522733 0.062756926 0.0980721 -0.16778782 0.144286 -0.006168606 -0.035547387 -0.11329139 0.031911355 0.0489303 -0.014881953 0.08786392 -0.16226773 0.14748363 -0.034039494 -0.15625799 -0.124049895 -0.051168893 -0.16651365 0.0245879 -0.09127846 0.16853853 -0.1322974 0.019345365 0.16355476 0.14485784 -0.094041094 0.07493519 0.16194187 -0.11533272 0.10439533 -0.045158364 -0.0459666 0.06296137 -0.08463926 -0.06924031 0.12824605 0.117082916 -0.032372344 0.07199175 0.11663326 -0.095977776 -0.0492278 -0.13361187 -0.1646763 0.11894446 0.16040151 0.14012027 -0.0064992085 -0.0554516 -0.113467455 -0.01803783 0.14850299 0.11050422 0.08632584 0.005759311 0.10048218 0.08039364 -0.047310818 -0.0309674 -0.009272257 -0.08245938 -0.008105995 -0.16565928 -0.031637553 0.16849276 0.10880279 -0.052484017 0.10228077 -0.1491425 -0.15229431 -0.020154191 0.04599964 -0.051803038 -0.15903258 0.12397759 0.024030453 -0.051416222 -0.15902077 0.043523807 0.12837914 -0.030206954 0.07239763 0.041454725 -0.13701935 -0.03932225 -0.043469932 -0.16379409 0.04313103 0.017000267 -0.041622143 -0.011262359 0.013028425 -0.089025855 -0.13297936 -0.07559678 -0.12670745 0.124692075 0.030822769 -0.078399 0.069145076 -0.043239508 -0.0328323 -0.11191149 0.11929868 -0.13432933 0.03446873 0.116868705 -0.1214929 0.14878006 -0.0058291135 -0.096431784 0.16296361 -0.008685943 -0.13292824 -0.14971183 -0.0010796139 -0.035200488 -0.09043802 -0.13619979 -0.06820556 0.15436147 -0.08860942 -0.009788058 -0.16897754 0.05955355 0.022277754 -0.03341066 0.07679744 0.11854094 -0.114918254 -0.16179152 0.035559066 -0.05735172 0.038512714 -0.11753878 0.118576705 0.07117676 -0.11415956 0.04770235 0.10045205 -0.1438506 0.16963667 -0.040348705 -0.042556677 -0.08289163 0.037767425 -0.16037084 -0.11538928 -0.030792719 0.14079732 -0.029725352 -0.034868624 0.16470963 -0.14678815 -0.09311681 -0.1580571 0.0057641733 0.0934194 0.077587895 0.01807445 0.1477784 0.13091813 0.011456245 0.11640125 0.040027298 0.11810414 0.09472712 -0.15234914 -0.043062326; 0.15044618 -0.08974225 0.08901625 0.12268592 0.06479537 0.061170567 -0.1539234 -0.11469982 -0.1289981 0.07521666 -0.10095314 0.078803174 -0.046903513 0.026280785 0.0032425853 -0.12994392 -0.07035343 -0.07826454 -0.06945804 -0.15343249 -0.16293448 0.099801 0.039952043 -0.024876978 0.040710006 0.062106933 0.16377434 0.043009777 -0.052012037 0.113842934 0.09241132 0.1277005 -0.16037422 -0.15041946 -0.14685406 -0.081566945 0.12320363 -0.13022716 0.12618816 0.08694927 0.16942096 0.040811423 -0.061506275 -0.055083137 -0.13018548 -0.011327259 -0.02039019 -0.109477706 0.1285171 0.06447038 -0.0796557 -0.03304995 0.16505012 -0.081258595 0.101121 0.057082027 -0.08043846 0.021986987 -0.023851907 -0.103796095 0.044273958 0.06222536 0.047551923 0.08177843 0.04525097 0.15361954 -0.034433022 -0.1344312 -0.15001212 0.13401122 0.04587212 -0.06354071 0.0026876011 -0.0056102043 -0.12893848 -0.16360456 0.027789326 -0.13506258 -0.029474035 0.06094584 0.012509288 0.025105225 -0.07772347 0.10713627 -0.013740186 0.12776604 -0.048382454 -0.14607413 -0.15629584 0.10603654 0.10874735 0.13086626 -0.015454738 -0.087439306 -0.10455882 -0.0199931 -0.04814021 0.009688796 0.15702328 -0.07326073 -0.023557255 -0.16600692 -0.016222427 0.02992581 0.09742868 0.11096049 -0.10297268 0.04302135 -0.037588026 -0.10985189 0.010206935 0.010512249 0.122113034 0.11096062 0.16989055 0.02246869 -0.1474917 0.010626423 0.08545045 -0.008543936 -0.013567459 -0.08284685 -0.011895935 -0.06583932 0.10760703 0.030757055 0.1508671 -0.113017835 0.09212651 -0.09826973 -0.063916296 -0.12294654 0.09909603 -0.024006976 -0.11998627 0.15057833 0.1478584 0.11773637 -0.06450084 -0.013007999 -0.09733688 -0.14434448 0.046617873 -0.14525081 -0.044492703 -0.026608314 -0.04517279 -0.13293664 0.06608149 0.06314057 -0.1512279 0.0007357675 -0.13844348 0.09236233 -0.05915186 0.09771499 -0.16133623 0.051902015 0.088049896 -0.049746815 0.044636175 -0.036114678 -0.010605671 0.012515331 -0.076398216 -0.12746081 -0.07583475 0.12796201 0.048227325 0.036899723 0.028073441 0.08336324 -0.09246859 0.01823204 -0.012760749 0.07487533 -0.03602679 0.13042666 0.093195446 -0.0146215 0.15351734 0.063388534 0.048456896 -0.1368913 0.0454403 -0.07142672 0.13683788 -0.020009946 -0.13219877 -0.103739314 0.11235936 0.10309483 0.053618215 0.041661732 -0.042290997 -0.15322118; -0.16356595 -0.025825571 -0.08743042 -0.121253245 0.07179276 -0.006706745 -0.09626418 0.16881442 -0.121828936 -0.08697266 -0.082036495 0.14175075 0.013858755 0.0852943 -0.116757706 0.0046635023 0.054822765 0.036758203 0.025748871 -0.15475287 0.0794672 0.009658787 0.083581135 -0.03192602 -0.0083100125 0.1568087 -0.028066443 -0.09798313 0.02446459 -0.07263019 -0.11321066 0.0012965092 -0.1601044 -0.02279502 0.035614807 0.049267817 0.077333994 0.053654652 -0.03554704 0.05747572 -0.09754432 -0.12789191 -0.008922226 0.13707542 0.10118889 -0.16880822 -0.030533222 -0.16902311 0.049765166 0.104236335 -0.009840099 -0.14923145 -0.16539545 -0.024804305 -0.1152157 0.050365843 0.14844018 -0.014392031 0.0065058204 0.098089576 0.10461342 -0.06626501 0.12118409 -0.028055212 0.12331683 0.0047055143 -0.017133912 -0.09082927 -0.0028588835 -0.11250083 0.021112712 -0.07462049 -0.105798036 -0.04464529 -0.09364376 0.03922765 0.08856093 -0.060048554 0.019103812 -0.13371131 -0.06002215 0.121490665 0.13741128 -0.16521993 0.056805827 0.15479346 0.11885515 -0.12767182 0.15266481 -0.10565729 -0.1563866 -0.09228518 -0.027340604 -0.07679301 0.0009744723 0.122414336 0.08807976 -0.06955073 0.10794556 -0.11590844 -0.07457232 0.15474251 -0.11318848 0.02403265 0.14875145 -0.10128244 0.030981416 -0.02274141 0.0815043 -0.07809273 0.05754186 0.09145243 -0.112009384 0.031863015 -0.040775638 -0.14938825 0.12784885 0.07230256 0.023874633 0.112100855 0.12775381 0.08985876 0.0033549492 -0.016364941 0.026320111 -0.15710297 -0.09811492 0.0029457351 0.1688089 -0.14683966 -0.054041915 0.0032419139 -0.070438795 -0.12410368 0.09279677 -0.0044522225 0.060069736 0.07730966 0.07808787 -0.108829565 -0.06636252 0.09764059 0.16028039 -0.12876911 -0.07919924 -0.09481562 -0.1050089 -0.08261793 0.09225733 0.12814084 0.1233728 0.10921615 0.15374538 -0.04288319 -0.05524567 -0.1689374 0.1596979 0.009961659 -0.042127017 0.0024801255 0.11665991 -0.15049988 -0.16705206 -0.09803302 0.055329308 -0.06232149 -0.15013096 -0.15932204 -0.09962206 0.021637464 -0.0733153 0.045207433 -0.14406271 0.065203995 -0.064992696 -0.01802794 -0.055695105 0.14043257 -0.15045667 -0.11208627 -0.09183967 -0.00014345076 -0.1587373 -0.036931887 0.13649009 0.11992831 0.120912224 0.16911097 -0.013793652 -0.16510299 0.01577647 -0.08943598 -0.14440541 0.16413161 -0.07741766 -0.037912056; -0.14190342 0.15597562 -0.020852381 0.14547507 0.029765554 0.037068482 -0.14471781 0.040524382 0.08265913 -0.06410158 -0.16735141 0.16143942 0.10176719 0.107900746 0.10656186 -0.09054754 -0.0416314 0.13430305 -0.031411424 0.06977114 -0.086715445 -0.11594044 -0.14763778 -0.115664035 0.015016839 0.16342095 0.08298934 0.14406572 -0.10149752 -0.12566721 0.1414266 -0.16070847 0.029296342 -0.01601597 -0.057687182 0.00032348136 0.04192011 -0.09805788 0.049943566 0.1537806 0.12465309 -0.006462954 0.11331776 0.15316513 0.028477265 -0.10933533 0.11191295 -0.15924849 -0.11467396 -0.025481237 -0.092574015 -0.14612956 -0.12492531 0.15594558 0.15606011 -0.13318856 -0.00025670993 -0.08892596 -0.11494568 0.059572652 -0.09294834 -0.16690105 -0.00090881984 -0.10381365 0.14556627 0.13681361 -0.13815248 0.100766435 0.024200454 -0.034977064 -0.023744142 -0.014229823 -0.11657967 -0.16714676 -0.1357053 0.1395914 0.11362582 -0.046119448 0.02696158 0.0014173366 -0.1545007 0.05672915 0.034175765 -0.048180982 0.07883548 0.12335941 0.14339389 0.10067207 0.11614783 -0.08693978 0.100404054 -0.041028786 -0.10521386 -0.038482726 -0.025599988 -0.06834223 -0.10593042 -0.14852968 0.028912153 -0.117388636 -0.14407904 -0.084607646 -0.13211398 0.08652618 -0.078909285 0.028706204 -0.021329405 0.07422872 -0.13411164 -0.027171051 0.04573363 -0.057092953 0.13599437 0.12553234 -0.028110102 0.11784845 0.020800238 0.047494978 0.0047304165 -0.10038546 -0.12728937 -0.09550378 -0.14587459 -0.020089515 0.16340223 0.043079395 -0.15373291 0.054595616 -0.13794114 0.13615766 -0.16951492 -0.1308925 -0.104257636 0.13608088 -0.0674278 0.032128394 0.11236011 -0.08804884 0.13536857 -0.14902271 0.108922765 -0.14447835 0.13702819 -0.012839606 -0.113915324 -0.05864135 -0.15085828 -0.15245563 -0.11590286 -0.12515676 0.06647027 0.020290928 0.16527496 0.09013576 -0.09802285 0.08761686 0.10664749 -0.028059443 -0.15154536 0.13276929 0.008188209 -0.081394866 -0.06963323 -0.050347857 0.059419822 0.00071953237 -0.06364469 0.090944685 -0.003550503 0.11652901 -0.08619387 -0.086498365 -0.122673124 -0.07008638 0.1067297 0.06884503 -0.04909277 -0.08826893 -0.08757786 -0.1255599 0.1589013 -0.16123727 0.11602548 0.12118293 0.10869252 0.011530666 -0.025517492 -0.021371722 -0.07056658 -0.0040332633 0.037025698 -0.16730078 -0.013563329 -0.08642407 -0.05773119 0.074045815; -0.086267695 0.07160964 0.14276196 -0.056224924 -0.09415248 0.035040133 -0.013323505 -0.011084689 -0.10289321 0.13056621 -0.13835754 0.040116876 0.10026182 0.16992484 -0.15170804 0.06601327 0.12137903 0.05463354 -0.051965978 0.019487636 -0.044823002 0.14456257 -0.14790544 -0.11070384 -0.10333299 -0.065083876 -0.05101132 -0.06748553 -0.09730193 -0.04462696 0.16103913 -0.1353434 0.11279604 -0.11453108 0.14927134 -0.12316717 0.124405615 0.13749447 -0.11663812 -0.016223831 -0.06105733 -0.1297485 -0.116808 -0.05837089 -0.076748736 -0.1551372 0.072859354 -0.0055019497 -0.14524719 0.14045168 0.073765814 -0.123661004 0.15638554 0.06844882 -0.06130877 0.14405198 0.12350508 -0.008827562 -0.095994495 -0.057441216 -0.094185054 -0.04313286 -0.056632716 0.11576311 0.13175696 0.1473571 -0.1274756 -0.10536384 -0.15879126 0.048823446 0.14709157 0.07884976 0.14410433 0.0060750004 -0.114165686 0.023347683 0.14573453 -0.0720926 -0.038169906 -0.06702552 -0.066770785 0.004139239 0.13849978 0.14139341 -0.16732985 -0.070857346 0.02507365 -0.1005129 -0.09668046 0.13140535 -0.021193217 0.10201695 0.134828 0.15612157 0.116644815 0.06508008 0.12629734 0.11087647 0.051888462 0.13936362 -0.049377985 -0.0019109204 -0.03316571 -0.007912151 0.05830015 0.023565514 -0.1611672 0.03621685 -0.06455494 0.088111766 0.04949124 -0.16856787 0.14575568 0.15981387 -0.12688634 0.15312667 -0.028365348 0.085679695 0.09825754 -0.15425357 0.16381846 0.064153455 -0.011757184 0.07878545 0.012116553 0.14453535 -0.12241971 0.09594921 -0.15179479 -0.0662915 -0.07685227 0.14556023 -0.09794258 0.12694876 -0.036300406 -0.008990869 0.1588909 0.06795165 0.065411896 -0.14570406 -0.10543293 -0.10186849 0.07669582 0.16333856 -0.084971204 -0.14217947 -0.03894616 -0.09154921 -0.088848546 -0.11518939 0.08412525 0.10059131 -0.015054904 -0.16528182 -0.13393772 -0.07190057 0.109388575 0.06574498 0.04712472 0.13303322 -0.019846762 -0.15506181 -0.088670515 -0.050762564 -0.006997003 -0.12482824 0.063552 0.09724995 -0.12628354 -0.062396847 -0.009914541 -0.026073594 0.06696072 0.122096024 0.12543876 0.018207544 -0.065850265 0.03079445 0.05009829 -0.081772834 0.11783854 -0.15051946 0.058952197 -0.08167239 0.044929527 -0.15512328 0.059637997 -0.07774717 -0.061274752 0.035674684 -0.11924585 0.1548953 0.13816234 -0.12819052 0.13788822 -0.09846445; 0.049161945 -0.0648809 0.0759758 0.14509414 -0.13156813 0.09019519 -0.15321594 -0.10625176 -0.06684278 -0.15040883 0.009502031 0.07439298 0.023342008 -0.0062242695 -0.08480853 -0.1039487 -0.034045536 0.11403326 0.04253491 0.042289004 0.03441404 -0.039116666 0.041012574 -0.019296518 -0.14028144 0.14724529 0.047718484 -0.060005467 -0.13766612 -0.09410972 -0.0748191 -0.03901832 0.03443793 -0.1450795 0.029331254 0.07031549 -0.07353097 0.026476542 0.06628566 0.08364042 -0.032858565 -0.09423061 0.06533127 -0.111246705 0.040585436 -0.11868256 0.021590935 -0.0015089286 0.16355287 -0.029437292 -0.0061259232 0.06492897 0.04121354 -0.14539881 0.1232866 -0.030239565 -0.15622628 0.15104884 0.027347704 -0.16650318 0.034990408 -0.040189344 0.054131653 -0.110656336 0.053198036 0.030391704 -0.15059696 -0.118828736 -0.06636206 0.12205621 -0.005612239 0.044006627 0.10785017 -0.108338706 0.09201999 0.08313548 -0.16569622 -0.15066248 0.09963348 0.16367126 0.022195643 0.13125613 -0.045087766 -0.120157026 0.01835777 -0.0591884 -0.16168354 -0.10639084 0.04497817 -0.15192191 -0.11629783 -0.09435979 0.13576637 0.10431161 0.08313851 0.16907574 0.019173777 0.14432643 -0.00629041 -0.036919884 0.13345192 -0.14242715 0.019510625 -0.13786575 0.12537651 -0.03039559 0.06239821 0.15211292 0.00880443 -0.13631745 0.06951563 -0.1353811 0.14947705 0.053665515 -0.06838282 -0.006400455 -0.09995971 -0.14834169 -0.16725218 0.14404741 0.053651337 -0.1663558 0.15916869 -0.13523188 -0.13449828 -0.09142321 -0.061060056 0.005908275 0.026474161 -0.00026932365 -0.058711704 -0.1337609 0.07457502 0.16046797 0.111121826 0.08620833 0.07906285 -0.016858364 -0.11397047 -0.15375842 0.041490044 0.09848961 0.05558317 -0.012499788 0.09856475 0.011009536 -0.12610066 -0.1700179 0.05340425 -0.03564862 -0.10124875 0.0009983571 0.10178113 -0.071018256 0.026158534 0.10904378 -0.16106208 0.11765731 0.06024248 0.11256089 -0.046419535 -0.15679318 0.071532674 -0.06537764 -0.16202766 0.124759555 -0.13701221 -0.0036158094 -0.080706485 0.057990707 0.003283946 -0.10897234 -0.0022395286 0.077199355 -0.07774828 0.13775966 0.06320779 0.092809625 0.06680008 0.13381445 0.0987936 0.0036709642 -0.05723551 -0.10446655 0.08113342 -0.10425108 0.08960452 0.033078186 -0.0032955427 0.15819243 -0.13806632 0.06508105 -0.15200984 0.030103581 -0.17001286 0.029212644]
b = Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
hw = Float32[-0.41776606 -0.15417005 -0.3138428 -0.11693898 -0.1442771 -0.38951784 0.36747855 0.2843635 -0.057276607 -0.23527922; 0.14475492 0.19248506 -0.30588362 0.44147548 -0.1538526 0.4781861 -0.3347122 -0.27019772 0.24485832 0.42609134; 0.397334 -0.26666638 0.2752305 -0.30962926 0.52155054 0.34879598 0.24329865 -0.16673236 0.08073458 -0.21562193; -0.047969393 -0.5292673 0.3447526 0.024188347 0.3285987 -0.031443577 -0.11880069 0.19138029 -0.38625902 0.52828693; 0.11626312 0.2928738 0.48581985 0.40742952 0.5361406 -0.42087412 0.38098827 0.14834109 -0.11608252 0.36018762; 0.40340298 -0.31209293 -0.15095611 0.5408967 0.33512765 -0.30299747 -0.09202581 -0.17228746 0.053287297 -0.0057894546; 0.036124542 -0.19110058 0.4592682 -0.3581527 0.026799243 0.3964407 0.426098 0.22990523 0.13915676 0.54177874; -0.06876593 -0.13380739 0.3974486 0.07639354 0.5204802 -0.5332434 0.26321465 -0.41810757 0.28973436 -0.108629584; 0.11724429 0.41077372 -0.31404802 0.29831648 -0.07813674 0.15723023 0.09238825 -0.3552328 0.5308343 -0.46524596; -0.10073735 0.34237245 -0.47963223 -0.17225514 0.18546012 0.47143182 -0.32338956 0.06846447 0.24077198 0.5279682]
state = Float32[0.9052552; -0.15950671; -0.57405686; -0.74819285; -0.8847531; 0.23532645; -0.8822605; 0.16834006; 0.26047626; -0.607836;;]
g = Float32[0.26084682; 0.08994258; 0.059419435; 0.049923174; 0.043550666; -0.86651295; 0.043659363; 0.12483817; 0.13688682; 0.057445813;;]

10×1 Matrix{Float32}:
  0.26084682
  0.08994258
  0.059419435
  0.049923174
  0.043550666
 -0.86651295
  0.043659363
  0.12483817
  0.13688682
  0.057445813

In [669]:
@show size(x)
@show size(w)
@show size(b)
@show size(hw)
@show size(state)
@show size(g)
zL = w * xT .+ hw * state .+ b
# h = tanh_deriv(zL)' * g
h = g .* tanh_deriv(zL)
new_w = w .* h
# we'll also have to calculate dx, but we can hide it in this exercise and not do it at all
# we most likely have to calculate the gradient relevant to previous state and add it to the next one
new_hw = hw .* h
new_b = sum(h, dims=2)

@show size(new_hw)
@show size(new_w)

@show sum(new_b)
@show sum(new_hw)
@show sum(new_w)

size(x) = (196, 1)
size(w) = (10, 196)
size(b) = (10,)
size(hw) = (10, 10)
size(state) = (10, 1)
size(g) = (10, 1)
size(new_hw) = (10, 10)
size(new_w) = (10, 196)
sum(new_b) = -0.25461861f0
sum(new_hw) = -0.16757038f0
sum(new_w) = 0.074533254f0


0.074533254f0

In [504]:
@show sum(h .* state)
@show sum(h * state')
@show size(h)
@show size(state)

sum(h .* state) = -0.1219393f0
sum(h * state') = 0.5823657f0
size(h) = (10, 1)
size(state) = (10, 1)


(10, 1)

In [670]:
state1 = Float32[0.80616355; -0.99324816; 0.999962; -0.9999927; 0.99999857; -0.07662304; 1.0; -0.6067625; -0.9999995; 0.8452436;;]
state2 = Float32[-0.97755146; -1.0; -0.9996698; 1.0; -1.0; 0.93019545; -0.994541; -0.9998532; -1.0; -1.0;;]
state3 = Float32[-0.998499; -0.9999859; 1.0; -1.0; 1.0; -1.0; 0.9999996; -0.9999996; -0.9579484; 0.9994851;;]
state4 = Float32[-0.98293906; -1.0; -1.0; 1.0; -1.0; -0.9765655; -0.99978566; 0.97993356; -1.0; -1.0;;]

states = [state1, state2, state3, state4]
result = zeros(Float32, size(hw))
prev = zeros(Float32, size(state1))
new_w = zeros(Float32, size(w))
new_hw = zeros(Float32, size(hw))
new_b = zeros(Float32, size(b))

for i in reverse(eachindex(states))
    state = states[i]
    zL = (w * xT .+ hw * state .+ b) + prev
    h = g .* tanh_deriv(zL)
    new_w .+= w .* h
    
    # we'll also have to calculate dx, but we can hide it in this exercise and not do it at all
    # we most likely have to calculate the gradient relevant to previous state and add it to the next one
    new_hw .+= hw .* h
    new_b .+= sum(h, dims=2)
end

@show sum(new_w)
@show sum(new_hw)
@show sum(new_b)


sum(new_w) = 0.4985255f0
sum(new_hw) = -0.5002134f0
sum(new_b) = -0.97787887f0


-0.97787887f0